In [44]:
import boto3
from dotenv import load_dotenv
import os
import warnings
from io import StringIO
import pandas as pd
from tqdm import tqdm
import numpy as np
import io
import requests
from datetime import datetime


import sys
sys.path.append('..')
import utils
tqdm.pandas()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
warnings.filterwarnings('ignore')

load_dotenv()
aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_access = os.getenv('AWS_SECRET_ACCESS')
aws_region = os.getenv('AWS_REGION')

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_access,
                region_name=aws_region)

bucket = 'footballbets'
league = "ENG-Premier League"
season = 2223

In [2]:
scheduler = s3.get_object(Bucket=bucket, Key=f'ENG-Premier League/2223/schedule.csv')
schedule = pd.read_csv(StringIO(scheduler['Body'].read().decode('utf-8')))

In [31]:
schedule.sort_values('start_time').head()

,Unnamed: 0,index,league,season,game,stage_id,ws_game_id,status,start_time,home_team_id,home_team,home_yellow_cards,home_red_cards,away_team_id,away_team,away_yellow_cards,away_red_cards,has_incidents_summary,has_preview,score_changed_at,elapsed,last_scorer,is_top_match,home_team_country_code,away_team_country_code,comment_count,is_lineup_confirmed,is_stream_available,match_is_opta,home_team_country_name,away_team_country_name,date,home_score,away_score,incidents,bets,aggregate_winner_field,winner_field,period,extra_result_field,home_extratime_score,away_extratime_score,home_penalty_score,away_penalty_score,started_at_utc,first_half_ended_at_utc,second_half_started_at_utc,stage,fbref_game_id,und_game_id
0,0,0,ENG-Premier League,2223,2022-08-05 Crystal Palace-Arsenal,20934,1640674,6,2022-08-05T20:00:00,162,Crystal Palace,1,0,13,Arsenal,2,0,True,True,2022-08-05 21:42:09Z,FT,1.0,True,gb-eng,gb-eng,43,True,False,False,England,England,2022-08-05 19:00:00+00:00,0,2,"[{'minute': '20', 'type': 1, 'subType': 1, 'pl...",NaN,NaN,1.0,7,NaN,NaN,NaN,NaN,NaN,2022-08-05T19:00:28Z,2022-08-05T19:46:51Z,2022-08-05T20:02:21Z,NaN,e62f6e78,18202
3,3,3,ENG-Premier League,2223,2022-08-06 Fulham-Liverpool,20934,1640675,6,2022-08-06T12:30:00,170,Fulham,2,0,26,Liverpool,0,0,True,True,2022-08-06 14:09:32Z,FT,1.0,True,gb-eng,gb-eng,41,True,False,False,England,England,2022-08-06 11:30:00+00:00,2,2,"[{'minute': '32', 'type': 1, 'subType': 1, 'pl...",NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,2022-08-06T11:30:29Z,2022-08-06T12:17:44Z,2022-08-06T12:33:43Z,NaN,6713c1dc,18203
1,1,1,ENG-Premier League,2223,2022-08-06 Bournemouth-Aston Villa,20934,1640676,6,2022-08-06T15:00:00,183,Bournemouth,3,0,24,Aston Villa,3,0,True,True,2022-08-06 16:39:30Z,FT,0.0,False,gb-eng,gb-eng,4,True,False,False,England,England,2022-08-06 14:00:00+00:00,2,0,"[{'minute': '2', 'type': 1, 'subType': 1, 'pla...",NaN,NaN,0.0,7,NaN,NaN,NaN,NaN,NaN,2022-08-06T14:00:14Z,2022-08-06T14:48:29Z,2022-08-06T15:03:32Z,NaN,877e3193,18204
4,4,4,ENG-Premier League,2223,2022-08-06 Leeds-Wolves,20934,1640677,6,2022-08-06T15:00:00,19,Leeds,2,0,161,Wolves,0,0,True,True,2022-08-06 16:34:30Z,FT,0.0,False,gb-eng,gb-eng,7,True,False,False,England,England,2022-08-06 14:00:00+00:00,2,1,"[{'minute': '6', 'type': 1, 'subType': 1, 'pla...",NaN,NaN,0.0,7,NaN,NaN,NaN,NaN,NaN,2022-08-06T14:00:48Z,2022-08-06T14:47:51Z,2022-08-06T15:05:00Z,NaN,82702941,18205
5,5,5,ENG-Premier League,2223,2022-08-06 Newcastle-Nottingham Forest,20934,1640679,6,2022-08-06T15:00:00,23,Newcastle,0,0,174,Nottingham Forest,3,0,True,True,2022-08-06 16:38:13Z,FT,0.0,False,gb-eng,gb-eng,4,True,False,False,England,England,2022-08-06 14:00:00+00:00,2,0,"[{'minute': '58', 'type': 1, 'subType': 1, 'pl...",NaN,NaN,0.0,7,NaN,NaN,NaN,NaN,NaN,2022-08-06T14:01:41Z,2022-08-06T14:48:45Z,2022-08-06T15:04:47Z,NaN,1ac96eb4,18206


In [59]:
schedule.start_time.dtype

dtype('O')

In [69]:
something = requests.get("https://fixturedownload.com/download/champions-league-2022-AtlanticStandardTime.csv")

In [85]:
europe = pd.read_csv(io.StringIO(something.text))

europe['date'] = europe.Date.apply(lambda x: str(x).split(' ')[0])
europe['date'] = europe.date.apply(lambda x: datetime.strptime(x, "%d/%m/%Y").date())
europe['time'] = europe.Date.apply(lambda x: str(x).split(' ')[1])
europe['time'] = europe.time.apply(lambda x: datetime.strptime(x, "%H:%M").time())

In [86]:
europe['league'] = 'Champions League'
europe['season'] = 2223

europe['game'] = europe.apply(lambda x: f"{x.date} {x['Home Team']}-{x['Away Team']}", axis=1)
europe['start_time'] = europe.apply(lambda x: f"{x.date}T{x.time}", axis=1)
europe = europe.rename(columns={'Home Team':'home_team', 'Away Team':'away_team'})

cols_to_keep = ['league', 'season', 'game', 'start_time', 'home_team', 'away_team']
nul_cols = schedule.columns.difference(cols_to_keep)

europe = europe.drop(europe.columns.difference(cols_to_keep), axis=1)
europe[nul_cols] = np.nan
europe.head()

,home_team,away_team,league,season,game,start_time,Unnamed: 0,aggregate_winner_field,away_extratime_score,away_penalty_score,away_red_cards,away_score,away_team_country_code,away_team_country_name,away_team_id,away_yellow_cards,bets,comment_count,date,elapsed,extra_result_field,fbref_game_id,first_half_ended_at_utc,has_incidents_summary,has_preview,home_extratime_score,home_penalty_score,home_red_cards,home_score,home_team_country_code,home_team_country_name,home_team_id,home_yellow_cards,incidents,index,is_lineup_confirmed,is_stream_available,is_top_match,last_scorer,match_is_opta,period,score_changed_at,second_half_started_at_utc,stage,stage_id,started_at_utc,status,und_game_id,winner_field,ws_game_id
0,Dinamo Zagreb,Chelsea,Champions League,2223,2022-09-06 Dinamo Zagreb-Chelsea,2022-09-06T13:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dortmund,Copenhagen,Champions League,2223,2022-09-06 Dortmund-Copenhagen,2022-09-06T13:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Salzburg,Milan,Champions League,2223,2022-09-06 Salzburg-Milan,2022-09-06T16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Celtic,Real Madrid,Champions League,2223,2022-09-06 Celtic-Real Madrid,2022-09-06T16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Leipzig,Shakhtar Donetsk,Champions League,2223,2022-09-06 Leipzig-Shakhtar Donetsk,2022-09-06T16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
final_sched = pd.concat([schedule, europe], ignore_index=True).sort_values('start_time')

In [89]:
final_sched.home_team.unique()

array(['Crystal Palace', 'Fulham', 'Bournemouth', 'Leeds', 'Newcastle',
       'Tottenham', 'Everton', 'Manchester United', 'Leicester',
       'West Ham', 'Aston Villa', 'Wolves', 'Arsenal', 'Brighton',
       'Manchester City', 'Southampton', 'Brentford', 'Nottingham Forest',
       'Chelsea', 'Liverpool', 'Dortmund', 'Dinamo Zagreb', 'Celtic',
       'Salzburg', 'Leipzig', 'Sevilla', 'Paris', 'Benfica', 'Ajax',
       'Frankfurt', 'Napoli', 'Atlético', 'Club Brugge', 'Barcelona',
       'Inter', 'Sporting CP', 'Plzen', 'Bayern', 'Marseille', 'Porto',
       'Leverkusen', 'Milan', 'Shakhtar Donetsk', 'M. Haifa', 'Juventus',
       'Copenhagen', 'Real Madrid', 'Rangers', 'Man City'], dtype=object)